## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
# Import the API key.
%load_ext dotenv
%dotenv
import os
gkey = os.getenv('G6_KEY')

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cape Town,ZA,-33.9258,18.4232,62.38,78,100,7.00,overcast clouds
1,College,US,64.8569,-147.8028,42.58,87,75,4.61,broken clouds
2,Esik,KZ,43.3552,77.4525,51.22,44,100,0.22,overcast clouds
3,Vostok,RU,46.4856,135.8833,44.56,39,13,5.75,few clouds
4,Nikolskoye,RU,59.7035,30.7861,50.81,75,53,11.79,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cape Town,ZA,-33.9258,18.4232,62.38,78,100,7.00,overcast clouds
5,Rikitea,PF,-23.1203,-134.9692,71.33,72,14,23.24,few clouds
13,Bethel,US,41.3712,-73.4140,60.66,75,75,8.05,broken clouds
14,Vilhena,BR,-12.7406,-60.1458,66.60,97,100,3.20,overcast clouds
20,Busselton,AU,-33.6500,115.3333,60.39,58,24,8.41,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   202
Country                201
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Simplify the new DataFrame as clean_df (identical to clean_travel_cities in the instruction of Module 6 assignment)
clean_df = preferred_cities_df[preferred_cities_df.replace('', np.nan).notnull()].dropna(how='any')
clean_df.count()

City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, weather description and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,62.38,overcast clouds,-33.9258,18.4232,
5,Rikitea,PF,71.33,few clouds,-23.1203,-134.9692,
13,Bethel,US,60.66,broken clouds,41.3712,-73.4140,
14,Vilhena,BR,66.60,overcast clouds,-12.7406,-60.1458,
20,Busselton,AU,60.39,few clouds,-33.6500,115.3333,
25,Ponta Do Sol,PT,73.00,few clouds,32.6667,-17.1000,
26,Kutum,SD,74.98,scattered clouds,14.2000,24.6667,
30,Powell River,CA,65.34,clear sky,49.8496,-124.5361,
34,Sorrento,IT,64.65,overcast clouds,40.6260,14.3744,
35,Raga,SS,68.49,overcast clouds,8.4596,25.6780,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,62.38,overcast clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
5,Rikitea,PF,71.33,few clouds,-23.1203,-134.9692,People ThankYou
13,Bethel,US,60.66,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
14,Vilhena,BR,66.60,overcast clouds,-12.7406,-60.1458,Hotel Mirage
20,Busselton,AU,60.39,few clouds,-33.6500,115.3333,Observatory Guest House
25,Ponta Do Sol,PT,73.00,few clouds,32.6667,-17.1000,Hotel do Campo
26,Kutum,SD,74.98,scattered clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
30,Powell River,CA,65.34,clear sky,49.8496,-124.5361,Rodmay Hotel
34,Sorrento,IT,64.65,overcast clouds,40.6260,14.3744,Hilton Sorrento Palace
35,Raga,SS,68.49,overcast clouds,8.4596,25.6780,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df.replace('', np.nan).notnull()].dropna(how='any')
print(hotel_df[hotel_df.replace('', np.nan).isnull()].count()['Hotel Name'], " empty rows were dropped.")
clean_hotel_df.head(10)

11  empty rows were dropped.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,62.38,overcast clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
5,Rikitea,PF,71.33,few clouds,-23.1203,-134.9692,People ThankYou
13,Bethel,US,60.66,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
14,Vilhena,BR,66.60,overcast clouds,-12.7406,-60.1458,Hotel Mirage
20,Busselton,AU,60.39,few clouds,-33.6500,115.3333,Observatory Guest House
25,Ponta Do Sol,PT,73.00,few clouds,32.6667,-17.1000,Hotel do Campo
26,Kutum,SD,74.98,scattered clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
30,Powell River,CA,65.34,clear sky,49.8496,-124.5361,Rodmay Hotel
34,Sorrento,IT,64.65,overcast clouds,40.6260,14.3744,Hilton Sorrento Palace
38,Christchurch,NZ,62.47,clear sky,-43.5333,172.6333,The George


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 20.0), zoom_level=1.8)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
display(fig)

Figure(layout=FigureLayout(height='420px'))